In [1]:
import hypernetx as hnx
import networkx as nx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import sys

## Load tables

In [2]:
dff = pd.read_pickle("biggerTrans.pkl")

In [3]:
dffwos2 = pd.read_pickle("biggerTransNoSars2.pkl")

In [4]:
dffwos = pd.read_pickle("biggerTransNoSars.pkl")

In [5]:
dffwoc = pd.read_pickle("biggerTransNoCov.pkl")

## Build the hypergraph

In [6]:
dff.shape

(9524, 169)

In [7]:
# now we use some (though not all) of the from_dataframe() arguments and
# let the function take care of the dataframe manipulation
Hf = hnx.Hypergraph.from_dataframe(dff, # the whole dataframe, b and p columns
                                                    #columns=human_b_cols, # choose specific columns
                                                    zsc='columns', # other option is 'rows'
                                                    absolute=True, # absolute value after z-score is taken
                                                    lower_thresh=2) # applies the > 2 threshold after zscore and absolute value)

# options that I used the defaults for:
# transpose = False: this will transpose the dataframe after z-score and absolute value, essentially creating the dual hypergraph. Instead we're taking the dual after the fact (below).
# name = None (string): If you want to give the resulting hypergraph a "name" attribute. Not necessary.
# key = None (function which evaluates True or False): This is for more complcated thresholding. If you're just doing z-score > some threshold you don't need to worry about this.
# rows = None (list of row names): If you want to use only a subset of the rows. This is done before taking z-score so your z-score will be relative only to those rows chosen.
# upper_thresh = None (number): You can have a maximum value for the the zscore if you want. You can use both upper_thresh and lower_thresh.

In [8]:
# running example here for large s value because it finishes relatively quickly. 
# Small s values take a long time on these large hypergraphs!
#betcen30 = hnx.s_betweenness_centrality(HfD, s=30)
#clocen30 = hnx.s_harmonic_closeness_centrality(HfD, s=30)

In [9]:
Hf.shape

(8010, 159)

In [10]:
HfD = Hf.dual()
HfD.shape

(159, 8010)

In [11]:
# now we use some (though not all) of the from_dataframe() arguments and
# let the function take care of the dataframe manipulation
Hfwos2 = hnx.Hypergraph.from_dataframe(dffwos2, # the whole dataframe, b and p columns
                                                    #columns=human_b_cols, # choose specific columns
                                                    zsc='columns', # other option is 'rows'
                                                    absolute=True, # absolute value after z-score is taken
                                                    lower_thresh=2) # applies the > 2 threshold after zscore and absolute value)

# options that I used the defaults for:
# transpose = False: this will transpose the dataframe after z-score and absolute value, essentially creating the dual hypergraph. Instead we're taking the dual after the fact (below).
# name = None (string): If you want to give the resulting hypergraph a "name" attribute. Not necessary.
# key = None (function which evaluates True or False): This is for more complcated thresholding. If you're just doing z-score > some threshold you don't need to worry about this.
# rows = None (list of row names): If you want to use only a subset of the rows. This is done before taking z-score so your z-score will be relative only to those rows chosen.
# upper_thresh = None (number): You can have a maximum value for the the zscore if you want. You can use both upper_thresh and lower_thresh.

In [12]:
Hfwos2.shape

(7960, 154)

In [13]:
Hfwos2D = Hfwos2.dual()

In [14]:
Hfwos2D.shape

(154, 7960)

In [15]:
graph = HfD

In [16]:
import ray

In [17]:
ray.init(num_cpus=5)

2020-05-23 13:48:29,024	INFO resource_spec.py:204 -- Starting Ray with 9.96 GiB memory available for workers and up to 5.0 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-05-23 13:48:29,408	INFO services.py:1168 -- View the Ray dashboard at localhost:8265


{'node_ip_address': '130.20.194.73',
 'raylet_ip_address': '130.20.194.73',
 'redis_address': '130.20.194.73:55826',
 'object_store_address': '/tmp/ray/session_2020-05-23_13-48-29_006939_51506/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-05-23_13-48-29_006939_51506/sockets/raylet',
 'webui_url': 'localhost:8265',
 'session_dir': '/tmp/ray/session_2020-05-23_13-48-29_006939_51506'}

In [25]:
@ray.remote
def betweenness(s, graph = graph):
    sbt = hnx.s_betweenness_centrality(graph, s=s)
    sbt = pd.Series(sbt)
    sbt.to_pickle("intermediateNoSars2/" + str(s) + "-betweenness.pkl")
    return(sbt)

@ray.remote
def closeness(s, graph = graph):
    scl = hnx.s_harmonic_closeness_centrality(graph, s=s)
    scl = pd.Series(scl)
    scl.to_pickle("intermediateNoSars2/" + str(s) + "-closeness.pkl")
    return(scl)

In [24]:
graph = Hfwos2D

In [ ]:
Hfwos2Dsbt = ray.get([betweenness.remote(i) for i in range(1,51)])

2020-05-23 14:14:48,715	WARNING worker.py:1090 -- The monitor failed with the following error:
Traceback (most recent call last):
  File "/Users/feng626/anaconda3/envs/network/lib/python3.8/site-packages/redis/connection.py", line 190, in _read_from_socket
    data = recv(self._sock, socket_read_size)
  File "/Users/feng626/anaconda3/envs/network/lib/python3.8/site-packages/redis/_compat.py", line 71, in recv
    return sock.recv(*args, **kwargs)
TimeoutError: [Errno 60] Operation timed out

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/feng626/anaconda3/envs/network/lib/python3.8/site-packages/ray/monitor.py", line 315, in <module>
    monitor.run()
  File "/Users/feng626/anaconda3/envs/network/lib/python3.8/site-packages/ray/monitor.py", line 260, in run
    self._run()
  File "/Users/feng626/anaconda3/envs/network/lib/python3.8/site-packages/ray/monitor.py", line 211, in _run
    self.update_raylet_map()
  File

(pid=raylet) E0523 15:23:57.143980 274400704 node_manager.cc:3537] Failed to send get core worker stats request: IOError: 14: Operation timed out
(pid=raylet) E0523 15:23:57.146301 274400704 node_manager.cc:3537] Failed to send get core worker stats request: IOError: 14: Operation timed out
(pid=raylet) E0523 15:23:57.146435 274400704 node_manager.cc:3537] Failed to send get core worker stats request: IOError: 14: Operation timed out
(pid=raylet) E0523 15:23:57.146550 274400704 node_manager.cc:3537] Failed to send get core worker stats request: IOError: 14: Operation timed out
(pid=raylet) E0523 15:23:57.146718 274400704 node_manager.cc:3537] Failed to send get core worker stats request: IOError: 14: Operation timed out
(pid=raylet) E0523 15:23:57.146823 274400704 node_manager.cc:3537] Failed to send get core worker stats request: IOError: 14: Operation timed out
(pid=raylet) E0523 15:23:57.146926 274400704 node_manager.cc:3537] Failed to send get core worker stats request: IOError: 14